In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate mediapy peft openai gtts pyttsx3 ffmpeg-python

In [ ]:
script = """scene_heading:INT. HOUSE (ENTRY LOBBY) -DAY"
action:A ghoulish figure is standing on stairs going to the basement looking towards the protagonist.
action:The wall above the ghoulish figure reads 'THIS PERSON IS NOT A TENANT IN THIS BUILDING'
action:Camera pans to the protagonist, standing by the door of the attached room, intially staring at the ghoulish figure and then slightly glancing at the text above.
action:Camera pans back to the ghoulish figure, the text on the wall has changed to 'THEY JUST KEEP STARING AT ME'
action:Camera pans back to the protagonist, he closes the door.
scene_heading:INT. HOUSE (ATTACHED ROOM) -DAY
action:The protagonist closes the door and looks through the peep hole and walks off.
scene_heading:INT. HOUSE (KITCHEN) -DAY
action:The protagonist walks through the kitchen to the another room.
scene_heading:INT. HOUSE (LIVING ROOM) -DAY
action:The protagonist walks towards the phone kept on the table, picks it up and heads back to the kitchen.
scene_heading:INT. HOUSE (KITCHEN) -DAY
action:The protagonist is calling somebody on the phone and looks towards his left.
action:Camera pans, the ghoulish figure is standing in the kitchen and staring at the protagonist.
action:The protagonist is holding the phone to his ear and staring at the ghoulish figure.
action:The ghoulish figure is staring at the protagonist,standing still.
action:The protagonist is still holding the phone to his ear, staring at the ghoulish figure and starts moving backwards to the living room.
scene_heading:INT. HOUSE (LIVING ROOM) -DAY
action:The protagonist is moving backwards holding the phone to his ear, moving backwards. he drops the phone on the floor.
action:The phone has fallen on the floor, and protagonist is walking fast towards another room.
scene_heading:INT. HOUSE (BEDROOM) -DAY
action:The protagonist is taking something out of his cupboard.
action:The protagonist has taken out a bag from the cupboard.
action:The bag is thrown on the floor and the protagonist takes out a hammer from the bag.
scene_heading:INT. HOUSE (LIVING ROOM) -DAY
action:The protagonist is holding the hammer in his hand and looking towards the other the kitchen.
action:POV of the protagonist looking towards the empty kitchen.
action:Camera pans to the protagonist still holding the hammer in his hand, slowly walking in the living room.
action:POV of the protagonist, camera pans to left of the protgonist in the living room.
action:The protagonist is looking towards his left with hammer still in the hand, the ghoulish figure's head is seen above the protagonist also peeking towards the same direction as him.
action:END CREDIT - BLACK SCREEN - 'TENANT' written on screen
"""

Call gpt and pass the script as context and ask for the director shots which will be needed to create an Image.

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY'],
)

chat_completion = client.chat.completions.create(
    messages=[
        {
          "role": "system",
          "content": """provided with a plot, generate a Shot script which includes shot description given by a the Director which should be in a format of a prompt to generate its equivalent shot image(should be below 77 tokens) which should look exactly like what a director would have imagined in his brain. Strictly respond in below format
                        [
                          {
                            "shot" : <Shot description>,
                            "image_prompt" : <Shot image prompt representing the entire shot precisely like a movie scene>
                          }
                        ]
                        make atleast 10-12 shots with each shot if shown for 10secounds should combined and make a video of 2mins(120secs)
                      """
        },
        {
            "role": "user",
            "content": script,
            "response_format" : {"type": "json_object"}
        }
    ],
    model="gpt-4-turbo",
)
shots = chat_completion.choices[0].message.content
print(shots)

In [ ]:
import mediapy as media
import random
import sys
import torch

from diffusers import DiffusionPipeline, TCDScheduler
from huggingface_hub import hf_hub_download

# Choose among 1, 2, 4 and 8:
num_inference_steps = 8

base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
repo_name = "ByteDance/Hyper-SD"
plural = "s" if num_inference_steps > 1 else ""
ckpt_name = f"Hyper-SDXL-{num_inference_steps}step{plural}-lora.safetensors"
device = "cuda"

pipe = DiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16, variant="fp16").to(device)
pipe.load_lora_weights(hf_hub_download(repo_name, ckpt_name))
pipe.fuse_lora()
pipe.scheduler = TCDScheduler.from_config(pipe.scheduler.config)

In [ ]:
seed = random.randint(0, sys.maxsize)

In [ ]:
import os
import json

output_folder = "output2"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
shots_list = json.loads(shots)
# Iterate over each object in the list
for i, shot in enumerate(shots_list):
    # Extract the image prompt from the shot
    prompt = shot["image_prompt"]

    # Set the eta value (adjust as needed)
    eta = 0.9

    # Generate the image
    images = pipe(
        prompt=prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=0.9,
        eta=eta,
        generator=torch.Generator(device="cuda").manual_seed(seed)
    ).images

    # Print the prompt and seed
    print(f"Prompt:\t{prompt}\nSeed:\t{seed}")

    # Save the generated image
    output_path = os.path.join(output_folder, f"output_{i}.jpg")
    images[0].save(output_path)

this for video generation from all the images that was generated. with audio in it

In [ ]:
import os
from moviepy.editor import ImageClip, concatenate_videoclips
from moviepy.video.fx.all import fadein, fadeout
from gtts import gTTS
import ffmpeg

# Set the input and output paths
input_folder = "output2"
output_video = "video.mp4"
audio_file = "narration.mp3"

# Set the desired duration for each image (in seconds)
duration = 6

# Get the list of image files in the input folder
image_files = [f for f in os.listdir(input_folder) if f.endswith(".jpg")]

# Sort the image files based on their names
image_files.sort()

# Create a list of image clips with fade-in and fade-out transitions
clips = []
for image_file in image_files:
    clip = ImageClip(os.path.join(input_folder, image_file)).set_duration(duration)
    clip = fadein(clip, 1)  # Fade-in duration of 1 second
    clip = fadeout(clip, 1)  # Fade-out duration of 1 second
    clips.append(clip)

# Concatenate the image clips
video_clip = concatenate_videoclips(clips)

# Set the text for narration
narration_text = """In this chilling tale, a protagonist finds himself in a haunting encounter with a ghoulish figure in his house. The eerie figure, standing on the basement stairs, bears a message above its head: "THIS PERSON IS NOT A TENANT IN THIS BUILDING." As the protagonist's gaze shifts between the figure and the message, the text mysteriously changes to "THEY JUST KEEP STARING AT ME," intensifying the unnerving atmosphere.
Seeking refuge, the protagonist closes the door and cautiously peeks through the peephole before moving through the kitchen and into the living room. With trembling hands, he picks up the phone, attempting to call for help. However, as he returns to the kitchen, he is confronted by the ghoulish figure, now standing motionless and staring directly at him.
Gripped by fear, the protagonist retreats to the living room, his phone falling to the floor as he hastily makes his way to the bedroom. Desperate for protection, he rummages through his cupboard, retrieving a bag containing a hammer. Armed with the tool, he ventures back into the living room, scanning the now empty kitchen with apprehension.
As the protagonist slowly moves through the living room, a chilling revelation unfolds. The ghoulish figure's head emerges above him, peeking in the same direction, as if mirroring his every move. The story concludes with a black screen and the ominous word "TENANT," leaving the viewer with a lingering sense of unease and the realization that the true nature of the protagonist's haunting experience remains a mystery."""


# Generate audio from the text using gTTS
tts = gTTS(text=narration_text, lang='en')
tts.save(audio_file)

# Write the video clip to a temporary file
video_clip.write_videofile("temp_video.mp4", fps=24, codec='libx264')

# Use ffmpeg-python to combine the video and audio files
input_video = ffmpeg.input("temp_video.mp4")
input_audio = ffmpeg.input(audio_file)
output = ffmpeg.output(input_video, input_audio, output_video, vcodec='copy', acodec='aac', strict='experimental')
ffmpeg.run(output)

# Remove the temporary files
os.remove("temp_video.mp4")
os.remove(audio_file)